In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

plt.style.use(['ggplot', 'assets/class.mplstyle'])

red = '#E24A33'
blue = '#348ABD'
purple = '#988ED5'
gray = '#777777'
yellow = '#FBC15E'
green = '#8EBA42'
pink = '#FFB5B8'

# Bayesian Estimates

## 1. Bayes theorem:

$$p(A|B) = \frac{p(A\cap B)}{p(B)} \\
  p(B|A) = \frac{p(B\cap A)}{p(A)}$$
  
but $p(A\cap B) = p(B\cap A)$ so we can write:

$$ p(A|B) = \frac{p(B|A) p(A)}{p(B)} $$

In Bayesian speak, these terms are:

- $p(A|B)$: the posterior probability of $A$ given $B$. This our ultimate goal, to obtain this probability function
- $p(A)$: the prior probability of $A$, before we looked at the data
- $p(B|A)$: the likelihood of $B$ given $A$
- $p(B)$: the probability of $B$ independent of $A$, or for all $A$. In general it's just a normalization hard to compute

In most useful applications for us:

- $A$ is a parameter (or parameter set) from a chosen distribution or a statistic
- $B$ de data observed

## 2. In Concrete.

Let's make it more concrete by looking at something we have already been calculating. Let's say we know that the outcomes from our experiment can be understood as RVs drawn from a gaussian with unkown $\mu$ and $\sigma$. Now we make the experiment a number of times and obtain a sample from the distribution of size $N$. This sample will be our data. What we will be calculating then is:

$$ p(\mu, \sigma| \{x_i\}_{i=1,...,N}) = \frac{p(\{x_i\}_{i=1,...,N}|\mu, \sigma) p(\mu, \sigma)}{p(\{x_i\}_{i=1,...,N})} $$

Notice that $p(\mu, \sigma| \{x_i\}_{i=1,...,N})$: the probability density associated to the probability that the parameters of the parent gaussian are a given value of $\mu$ and $\sigma$, given that we know a sample from said parent gaussian contains the $N$ values $\{x_i\}_{i=1,...,N}$; **IS A FUNCTION**. In this case it is a bivariate function that we need to know for all values of $(\mu, \sigma)$.

Several problems arise when trying to evaluate the right hand side of the expression:

- $p(\{x_i\}_{i=1,...,N}|\mu, \sigma)$: is the probability that we have of obtaining a given sample $\{x_i\}_{i=1,...,N}$ if we knew that the parent gaussian distribution has the known parameters $\mu$ and $\sigma$. This is called the likelihood and it is imperative that we know how to calculate it or get a very good approximation. If our model is a gaussian, then it is fairly simple.

- $p(\mu, \sigma)$ is the probability that the parameters of the parent gaussian take the given values $\mu$ and $\sigma$. Notice that this is a probability independent of any data set. It should be set before knowing anything about our samples. This part is controversial and can cause problems that we will explore later.

- $p(\{x_i\}_{i=1,...,N})$: finally, this is the probability of getting the given data set for any arbitrary gaussian (any arbitrary set of parameters). This one is very hard to compute but we will see that we do not need to compute it to get samples that follow the right distribution. Once we can draw samples that follow a distribution, we can compute statistics on them.

In our case, the likelihood is not hard to compute. If we know that the parent distribution for each data point in the sample is the same and that $x_i \sim \mathcal{N}(\mu, \sigma)$ for all $i$, then the likelihood of getting a given value is:

$$ p(x_i|\mu, \sigma) = \frac{1}{\sigma \sqrt{2\pi}} \exp\left(\frac{-(x_i-\mu)^2}{2\sigma^2}\right)$$

If we also know that the samples were obtained independently, then we can use the definition of independence to write that:

$$ p(x_1, x_2, ..., x_N|\mu, \sigma) = \prod_{i=1}^{N} \frac{1}{\sigma \sqrt{2\pi}} \exp\left(\frac{-(x_i-\mu)^2}{2\sigma^2}\right)$$

### 2.1. 1D Example

This is not a very realistic example but it helps for educational purposes. Let's restrict our discussion above a bit further. Let's say that we know our outcomes are RVs drawn from a gaussian with unknown $\mu$ but **known $\sigma$**. So, what we want is to calculate the posterior: 

$$ p(\mu|\{x_i\}_{i=1,...,N}|\sigma) = \frac{p(\{x_i\}_{i=1,...,N}|\mu|\sigma) p(\mu| \sigma)}{p(\{x_i\}_{i=1,...,N}|\sigma)} $$

The expression for the likelihood doesn't change.

We can treat the denominator as a normalizing constant but we need to define a prior:

$$ p(\mu|\{x_i\}_{i=1,...,N}) \propto p(\{x_i\}_{i=1,...,N}|\mu) p(\mu) $$

And we dropped $\sigma$ from the expression to avoid repetition but it should be understood that $\sigma$ is now a fixed value in the problem.

### 2.1.1. Analytic Expression

If we know nothing about the problem (e.g., if we have never made this experiment before), then we cannot constrain $\mu$ a priori. Problem is, we cannot represent $p(\mu)$ with a uniform distribution from $-\infty$ to $\infty$ because that cannot be normalized. But we can cheat and just assign a constant value to the prior, to represent equal belief in any value. If we then try to maximize the posterior, we can do it and the constant chosen vanishes. The trick is to maximize the $\log$ of the posterior. For the maximum a posteriori (MAP), we get the standard expression:

$$ \mu_{\rm MAP} = \frac{1}{N} \sum_{i=1}^{N} x_i $$

Two important points are that:

- In this case the $\mu_{\rm MAP}$ coincides with previous expressions derived from maximum likelihood or other methods but this is not generally the case as it should depend on the prior chosen. In any case, $\mu_{\rm MAP}$ is interesting but it is not the objective of Bayesian anaylsis, remember that we are after **a function $p(\mu|\{x_i\}_{i=1,...,N})$**.
- This is a case where an analytic expression for the posterior can be written. We can even chose a prior that is also gaussian and the full analytic expression is simple. This is something that sometimes is desirable but it is not general. For the general case we will need numerical methods and they have been developed extensively and are continouously been refined as this is an active are of mathematical research.

### 2.1.2. More General Methodology

So, what we are after is a function. The function takes $\mu$ as its argument and it's supposed to be a PDF so it should integrate to 1 over its full domain.

In general, it can be hard to derive said function. However, it is not impossible to take random samples from said PDF if we are able to evaluate a function that is proportional to the PDF (i.e., we do not need to normalize it). If we can create a large sample, we can create its normalized histogram and get a pretty good approximation of the PDF. **That's what MCMC methods are all about. Sampling unnormalized distributions**.